In [1]:
import sys
import os
import pandas as pd 
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.models.sasrec.ssept import SSEPT
from recommenders.models.sasrec.sampler import WarpSampler
from util_seren import SASRecDataSet
from recommenders.utils.notebook_utils import store_metadata
from recommenders.utils.timer import Timer

from model_seren import SASREC 

print(f"System version: {sys.version}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.9.19 (main, Mar 21 2024, 17:21:27) [MSC v.1916 64 bit (AMD64)]
Tensorflow version: 2.12.0


In [2]:
num_epochs = 10
batch_size = 128
seed = 100  # Set None for non-deterministic result

# data_dir = os.path.join("tests", "recsys_data", "RecSys", "SASRec-tf2", "data")
data_dir = os.path.join("./")

# Amazon Electronics Data
dataset = "reviews_Books_5_full"

lr = 0.0001  # learning rate
maxlen = 150  # maximum sequence length for each user
num_blocks = 3  # number of transformer blocks
hidden_units = 128  # number of units in the attention calculation
num_heads = 2  # number of attention heads
dropout_rate = 0.2  # dropout rate
l2_emb = 0.001  # L2 regularization coefficient (adjusted to 0.001)
num_neg_test = 20  # number of negative examples per positive example
model_name = 'sasrec'  # 'sasrec' or 'ssept'

In [3]:
inp_file = os.path.join(data_dir, dataset + ".txt")
print(inp_file)

# initiate a dataset class 
data = SASRecDataSet(filename=inp_file, col_sep="\t")

# create train, validation and test splits
data.split()

# some statistics
num_steps = int(len(data.user_train) / batch_size)
cc = 0.0
for u in data.user_train:
    cc += len(data.user_train[u])
print('%g Users and %g items' % (data.usernum, data.itemnum))
print('average sequence length: %.2f' % (cc / len(data.user_train)))

./reviews_Books_5_full.txt
2346 Users and 113876 items
average sequence length: 110.97


In [4]:
model = SASREC(item_num=data.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [128, 128],
               l2_reg=l2_emb,
               num_neg_test=num_neg_test
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer)

In [5]:
sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, batch_size=batch_size, maxlen=maxlen, n_workers=3)


In [6]:
with Timer() as train_time:
    results = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=10)

print('Time cost for training is {0:.2f} mins'.format(train_time.interval/60.0))

Evaluating...



epoch: 10, time: 245.8755755, valid: {'HT': {5: 0.4019607843137255, 10: 0.5302642796248934}, 'NDCG': {5: 0.2976117364981973, 10: 0.3387590662316224}, 'HRseren': {5: 0.5932203389830508, 10: 0.7288135593220338}, 'NDCGseren': {5: 0.472156784128777, 10: 0.5158637496066631}})
epoch: 10, time: 245.8755755,  test: {'HT': {5: 0.37382779198635974, 10: 0.5174765558397272}, 'NDCG': {5: 0.2748279422549639, 10: 0.32102331566700787}, 'HRseren': {5: 0.5416666666666666, 10: 0.7083333333333334}, 'NDCGseren': {5: 0.4091503022352871, 10: 0.46129883618845796}})


Results:
HT@5: 0.3832054560954817
HT@10: 0.5208866155157715
NDCG@5: 0.2777044150803018
NDCG@10: 0.3215899861589714
HRseren@5: 0.5208333333333334
HRseren@10: 0.7291666666666666
NDCGseren@5: 0.42233176953473245
NDCGseren@10: 0.4870232199739027
Time cost for training is 9.24 mins


In [7]:
print(results)

{'HT': {5: 0.3832054560954817, 10: 0.5208866155157715}, 'NDCG': {5: 0.2777044150803018, 10: 0.3215899861589714}, 'HRseren': {5: 0.5208333333333334, 10: 0.7291666666666666}, 'NDCGseren': {5: 0.42233176953473245, 10: 0.4870232199739027}}


In [8]:
model.evaluate_serendipity(dataset=data)

  0%|                              | 3/2346 [00:00<04:21,  8.97user/s]

user 1 processed. 134 interactions.
user 2 processed. 18 interactions.
user 3 processed. 82 interactions.
user 4 processed. 19 interactions.


  0%|                              | 5/2346 [00:00<03:54,  9.97user/s]

user 5 processed. 117 interactions.
user 6 processed. 13 interactions.


  0%|                              | 7/2346 [00:00<04:11,  9.28user/s]

...

user 2339 processed. 267 interactions.
user 2340 processed. 73 interactions.


100%|██████████████████████████▉| 2342/2346 [06:22<00:00,  7.03user/s]

user 2341 processed. 353 interactions.
user 2342 processed. 54 interactions.
user 2343 processed. 67 interactions.
user 2344 processed. 27 interactions.


user 2345 processed. 503 interactions.
user 2346 processed. 146 interactions.


Serendipity comparison saved
